In questo notebook considererò la configurazione "almostFull_35match_30min_5_(40%-0)" e andrò a salvare in un file i pattern verificati in un paio di situazioni in cui la predizione sarà "QuasiPiena" e quelli in un paio di situazione in cui la predizione sarà "Normal".

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_squared_error
from scipy.ndimage.interpolation import shift
from math import sin, cos, sqrt, atan2, radians 
from sklearn import tree, svm, linear_model, ensemble, neighbors, naive_bayes 
import dateutil
import os
from pathlib import Path
from joblib import dump, load
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import make_scorer
import ast


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
status_path = Path('../filtered_status.csv')
stations_path = Path('../station.csv')

In [3]:
status_df = pd.read_csv(status_path, parse_dates=['time'])
stations_df = pd.read_csv(stations_path)

In [4]:
#il seguente vettore è stato ottenuto in analisi fatte precedentemente. 
SFancisco_stations = [39, 41, 42, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58, 59, 60, 
                      61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 82]

### Calcolo la distanza tra le diverse stazioni della città di San Francisco

In [5]:
#function to retrieve distance between 2 stations
def getDistance(lat_a, long_a, lat_b, long_b):
    # approximate radius of earth in km
    R = 6373.0    
    lat1=radians(lat_a)
    lat2=radians(lat_b)
    lon1=radians(long_a)
    lon2=radians(long_b)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [6]:
coordinates_stations_df = stations_df[stations_df["id"].isin(SFancisco_stations)]
coordinates_stations_df = coordinates_stations_df[["id", "lat", "long"]]
records = coordinates_stations_df.to_records(index=False)
list_coo = list(records)
# print(list_coo)

voc_distances={}
for i in range(len(list_coo)):
    for j in range(i+1,len(list_coo)):
        station1=list_coo[i][0]
        station2=list_coo[j][0]
        lat_i=float(list_coo[i][1])
        long_i=float(list_coo[i][2])
        lat_j=float(list_coo[j][1])
        long_j=float(list_coo[j][2])
        distance=getDistance(lat_i, long_i, lat_j, long_j)
        id_stations=str(station1)+' '+str(station2)
        voc_distances[id_stations]=distance


In [7]:
# voc_distances

In [8]:
distance_dictionary = {}

for station_id in SFancisco_stations:
    distance_dictionary[station_id] = []
    
    delta0_stations = []
    delta1_stations = []
    delta2_stations = []

    for key in voc_distances.keys():
        if str(station_id) in key:
            distance = voc_distances[key]

            if key.split(" ")[0] == str(station_id):
                other=key.split(" ")[1]
            elif key.split(" ")[1] == str(station_id):
                other=key.split(" ")[0]
            
            if distance<=1:
                delta0_stations.append(other)         
            elif distance<=2:
                delta1_stations.append(other)
            elif distance<=3:
                delta2_stations.append(other)
        
    distance_dictionary[station_id].append(list(delta0_stations))
    distance_dictionary[station_id].append(list(delta1_stations))
    distance_dictionary[station_id].append(list(delta2_stations))


### Testo i pattern estratti

In [9]:
#Parametri considerati per la costruzione dei dataset della classificazione
time_interval = 30
window_width = 5

#Parametri usati per l'estazione dei pattern
space_interval = 1000

#Parametri usati per il filtraggio dei pattern
conf_threshold = 40 #valore intero (%)
sup_threshold = 0

#Flag utile a considerare soltanto i pattern contenenti esclusivamente QuasiPiena
filter_normal = True

#Numero di pattern da matchare
num_tot_pat = 35

In [11]:
#Se considero soltanto i pattern contenenti soltanto QuasiPiena
if filter_normal:
    pattern_path = f"./Results_extraction/{time_interval}min_{space_interval}m/filtered_results({conf_threshold}%-{sup_threshold})_StateChange_almostFull_{time_interval}_{space_interval}_0(3-3)_ordered_by_confidence.txt"
    output_path = f'Test_results/{time_interval}min/sample_matched_patterns/matched_patterns_almostFull_{num_tot_pat}match_{time_interval}min_{window_width}_({conf_threshold}%-{sup_threshold}).txt'
else:
    pattern_path = f"./Results_extraction/{time_interval}min_{space_interval}m/filtered_results({conf_threshold}%-{sup_threshold})_StateChange_Normal_almostFull_{time_interval}_{space_interval}_0(3-3)_ordered_by_confidence.txt"
    output_path = f'Test_results/{time_interval}min/sample_matched_patterns/matched_patterns_Normal_almostFull_{num_tot_pat}match_{time_interval}min_{window_width}_({conf_threshold}%-{sup_threshold}).txt'
    
file = open(output_path, "w")
file.write(f'MATCHED PATTERNS:\n\n')

#considero una stazione che ha avuto nel test una precisione abbstanza alta (92%)
station_id = 61 

normal_situation = 0
critic_situation = 0


test_path = Path(f'../Traditional_classification/datasets/{time_interval}_{window_width}/station{station_id}_test.csv')
test_df = pd.read_csv(test_path)


#itero le righe del test set
for index, row in test_df.iterrows():
    matched_pattern_list = []
    num_pat_matched = 0
    prediction = "N"

    #apro il file ed itero su tutti i pattern
    with open(pattern_path, "r") as ifile:
        for line in ifile:
            #elimino gli header del file
            if "[[[" not in line:
                continue

            flag = '' # sarà usaro per la condizione di uscita se una regola non metcha

            #Trasformo la linea testuale in una lista
            line_list = ast.literal_eval(line)

#             #Prelevo il conseguente
#             last = line_list[0][0][-1]

            #elimino il conseguente
            prior = line_list[0][0][:-1]

            #Prelevo l'ultimo elemento
            last = prior[-1]

            #Ricavo l'istante temporale dell'ultimo slot che corrisponderà all'istante 0 nella tabella del test set
            curr_time_slot = int(last[0].split('_')[1][1])


            #Itero gli slot di tempo precedenti al conseguente
            for item in prior:
#                 print(f"1_{flag}")

                matched_neigh = 0

                #controllo se è già stata settata la condizione di uscita
                if flag == "next_line":
                    break

                #ricavo la stringa contenuta nell'i-esimo slot e la splitto per ottenere la lista dei controlli da fare
                check_list = item[0].split(",")

                #itero sulla lista contenuta nello stesso slot temporale
                for el in check_list:
#                     print(f"2_{flag}")
                    #controllo se è già stato settata la condizione di uscita
                    if flag == "next_line":
                        break

                    el_status = el.split('_')[0]
                    el_time_slot = int(el.split('_')[1][1])
                    el_space_slot = int(el.split('_')[2])     

                    feature_time_slot = curr_time_slot - el_time_slot

                    #se la stazione è quella di riferimento
                    if el_space_slot == 0:

                        #A partire da queste info costruisco la feature da andare a controllare
                        feature_str = f"docks_av_{station_id}_T{feature_time_slot}"

#                         print(feature_str)

                        #Se la regola non è verificata, passo alla prossima
                        if (el_status == "QuasiPiena" and row[feature_str] > 2) or (el_status == "Normal" and row[feature_str] <= 2):
                            flag = "next_line"
#                             print(f"3_{flag}")
                            break


                    #se invece si tratta di una delle stazioni vicine:
                    else:
                        neighbors = distance_dictionary[station_id][el_space_slot-1]
                        for neigh_id in neighbors:
                            #inizialmente lo stesso al caso negativo
                            flag = "next_line"

                            #se il vicino corrente ha già verificato la regola, passo al vicino successivo 
                            # (utile nei casi in cui si hanno piu regole nello stesso delta spaziale)
                            if neigh_id == matched_neigh:
                                continue

                            feature_str = f"docks_av_{neigh_id}_T{feature_time_slot}"
#                             print(feature_str)

                            #se il vicino corrente non rispetta il pattern, passo al vicino successivo
                            if (el_status == "QuasiPiena" and row[feature_str] > 2) or (el_status == "Normal" and row[feature_str] <= 2):
                                continue

                            #se invece viene trovato un vicino che rispetta il pattern, rimetto il flag ='' ed esco dal ciclo  
                            flag=''
                            matched_neigh= neigh_id
                            break

                        if flag == "next_line":
                            break


            #se a questo punto il flag non è stato settato a "next_line", vuol dire che nessuna delle condizioni sfavorevoli 
            # precedenti è stata verificata, quindi il pattern ha metchato. 
            if flag != "next_line":
                #incremento il numero di pattern verificati
                num_pat_matched +=1
                matched_pattern_list.append(line)

                #se il numero di pattern verificati è uguale al numero di pattern da verificare desiderato
                if num_pat_matched == num_tot_pat: 
                    # Setto pred a "QP" ed esco
                    prediction = "QP"
    #                 print(f'PATTERN MATCHED NUM: {num_row}')
                    break
#     print (matched_pattern_list)
#     print (prediction)
    
    if prediction == "QP":
        critic_situation +=1
        if critic_situation <= 2 :
            file.write(f'Predicted status QUASI PIENA:\n')
            for pat in matched_pattern_list:
                file.write(pat)
            file.write('\n')
                
    elif prediction == "N" and len(matched_pattern_list) >5:
        normal_situation +=1
        if normal_situation <= 2 :
            file.write(f'Predicted status NORMAL:\n')
            for pat in matched_pattern_list:
                file.write(pat)
            file.write('\n')
    
    if critic_situation>2 and normal_situation>2:
        break
            
            
#     y_pred.append(prediction)

file.close()
